In [9]:
import pandas as pd
import sqlite3
orders_df = pd.read_csv("orders.csv")

print("Orders Data Loaded")
print(orders_df.head())

users_df = pd.read_json("users.json")

print("\nUsers Data Loaded")
print(users_df.head())

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

cursor.executescript(sql_script)

restaurants_df = pd.read_sql_query(
    "SELECT * FROM restaurants", conn
)

print("\nRestaurants Data Loaded")
print(restaurants_df.head())

merged_df = orders_df.merge(
    users_df,
    on="user_id",
    how="left"
)

final_df = merged_df.merge(
    restaurants_df,
    on="restaurant_id",
    how="left"
)

print("\nFinal Merged Dataset Preview")
print(final_df.head())

final_df.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)

print("\n Final dataset saved as 'final_food_delivery_dataset.csv'")


Orders Data Loaded
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  

Users Data Loaded
   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold

Restaurants Data Loaded
   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  

In [10]:
gold_city_revenue = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)
gold_city_revenue

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [11]:
cuisine_avg_order = (
    final_df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)
cuisine_avg_order

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [13]:
user_total_spend = (
    final_df
    .groupby("user_id")["total_amount"]
    .sum()
)
(user_total_spend > 1000).sum()

np.int64(2544)

In [14]:
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 – 5.0"]
)
final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)

/tmp/ipython-input-2089426043.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
4.6 – 5.0,2197030.75
4.1 – 4.5,1960326.26
3.0 – 3.5,1881754.57
3.6 – 4.0,1717494.41


In [15]:
(
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [16]:
(
    final_df
    .groupby("cuisine")
    .agg(
        restaurant_count=("restaurant_id", "nunique"),
        total_revenue=("total_amount", "sum")
    )
    .sort_values(by=["restaurant_count", "total_revenue"], ascending=[True, False])
)

,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [17]:
(
    final_df[final_df["membership"] == "Gold"].shape[0]
    / final_df.shape[0]
) * 100

49.87

In [28]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

(
    final_df[final_df["restaurant_name_x"].isin(options)]
    .groupby("restaurant_name_x")
    .agg(
        avg_order_value=("total_amount", "mean"),
        order_count=("order_id", "count")
    )
    .query("order_count < 20")
    .sort_values("avg_order_value", ascending=False)
    .head(1)
    .reset_index()
)

,restaurant_name_x,avg_order_value,order_count
0,Ruchi Foods Chinese,686.603158,19


In [19]:
(
    final_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [20]:
(
    final_df
    .assign(
        quarter=pd.to_datetime(final_df["order_date"]).dt.to_period("Q")
    )
    .groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

/tmp/ipython-input-2373803402.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  quarter=pd.to_datetime(final_df["order_date"]).dt.to_period("Q")


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [21]:
final_df[final_df["membership"] == "Gold"].shape[0]

4987

In [22]:
round(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
)


1889367

In [23]:
final_df["user_id"].nunique()

2883

In [24]:
round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(),
    2
)


np.float64(797.15)

In [25]:
final_df[final_df["rating"] >= 4.5].shape[0]

3374

In [26]:
top_gold_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_gold_city
final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_gold_city)
].shape[0]

1337